In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import Callback,EarlyStopping
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
train = "../input/100-bird-species/train"
valid = "../input/100-bird-species/valid"
test = "../input/100-bird-species/test"

In [ ]:
def proces_img(file_path):
    filepaths = list(glob.glob(file_path+'/**/*.jpg'))
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    img_df = pd.concat([filepaths, labels], axis=1)
    img_df['Label'] = img_df['Label'].apply(lambda x: np.NaN if x[-2:]=='GT' else x)
    img_df = img_df.dropna(axis=0)
    img_df = img_df.sample(frac=1).reset_index(drop = True)
    return img_df

In [ ]:
train_df = proces_img(train)
valid_df = proces_img(valid)
test_df = proces_img(test)

In [ ]:
sns.barplot(x=["Train","Valid","Test"],y=[len(train_df),len(valid_df),len(test_df)])
plt.title('Train, Valid and Test')

In [ ]:
train_df.shape

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_df.Filepath[i]))
    ax.set_title(train_df.Label[i])
plt.tight_layout()
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
                    preprocessing_function=preprocess_input,
                    validation_split=0.2)
test_datagen = ImageDataGenerator(
                    preprocessing_function=preprocess_input)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100,100),
    batch_size=32
)
valid_gen = train_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100,100),
    batch_size=32
)
test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(100, 100),
    batch_size=32
)

In [ ]:
pretrained_model = ResNet50(
    input_shape=(100, 100, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [ ]:
pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input

x = Dense(120, activation='relu')(pretrained_model.output)
x = Dense(120, activation='relu')(x)

outputs = Dense(300, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
my_callbacks  = [EarlyStopping(monitor='val_accuracy',
                              min_delta=0,
                              patience=3,
                              mode='auto')]

In [ ]:
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=my_callbacks
)

In [ ]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
results = model.evaluate(test_gen, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))